# DeepLabCut Notebook

Import python libraries needed for this notebook

In [ ]:
# Restart Kernel before running this cell if you've run other matplotlib commands
%matplotlib qt

In [ ]:
try:
    import deeplabcut
    import tkinter
    from tkinter import filedialog
    
    print(f'Using DeepLabCut version: {deeplabcut. __version__}')

except:
    print("Please run the notebook in in your local environment")

### You might have to import torch to use PyTorch engine ###

In [ ]:
import torch
print("PyTorch version:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))
    print("cuDNN Version:", torch.backends.cudnn.version())
    print("cuDNN Enabled:", torch.backends.cudnn.enabled)

### Check PyTorch in DeepLabCut ###

In [ ]:
if torch.cuda.is_available():
    print("DeepLabCut is using PyTorch with GPU support.")
else:
    print("DeepLabCut is using PyTorch on CPU.")

Start by selecting the list of videos to be included in the model. You could manually type the full path of each video in a python list as argument of the deeplabcut.create_new_project() function, like so:

:::{note}
Windows users need to use the double backslash for path directories or a python raw filestring.
:::

Instead, we use ```tkinter``` to open a file dialoge and save the file paths in a python list called ```videolist```: 

In [ ]:
video_files = filedialog.askopenfilenames(title='Choose new video files to analyze in DeepLabCut:')
videolist = list(video_files)

print(f'{len(videolist)} videos selected:')
for i in range(len(videolist)): 
    print(videolist[i])

Now we create a new project using the video paths in ```videolist```, give the project a name and set a few parameters:

## Create new DLC Project

In [ ]:
config_path = deeplabcut.create_new_project(
    'DLC-WhiteAnimals', 'Atanu', 
    [r'C:\DeepLabCutProjects\data\ToyOnlyWhite\SplitVideos\ToyOnly_2_25_25_S1P_Dallas.mp4'], 
    working_directory='C:/DeepLabCutProjects/', copy_videos=False, multianimal=False
)

:::{note}
You can load existing DeepLabCut projects by specifying the config_path as below:
:::

## Add new videos

In [ ]:
new_videos = [r"C:\DeepLabCutProjects\data\ToyOnlyWhite\SplitVideos\ToyOnly_3_4_25_S1Y_Austin.mp4",
              r"C:\DeepLabCutProjects\data\ToyOnlyWhite\SplitVideos\ToyOnly_3_4_25_S1Y_Berlin.mp4",
              r"C:\DeepLabCutProjects\data\ToyOnlyWhite\SplitVideos\ToyOnly_3_4_25_S1Y_Houston.mp4",
             r"C:\DeepLabCutProjects\data\ToyOnlyWhite\SplitVideos\ToyOnly_3_4_25_S1Y_Toronto.mp4",
             r"C:\DeepLabCutProjects\data\ToyOnlyWhite\SplitVideos\ToyOnly_3_4_25_S4Y_London.mp4",
             r"C:\DeepLabCutProjects\data\ToyOnlyWhite\SplitVideos\ToyOnly_3_4_25_S4Y_Paris.mp4",
             r"C:\DeepLabCutProjects\data\ToyOnlyWhite\SplitVideos\ToyOnly_3_4_25_S4Y_Phoenix.mp4",
             r"C:\DeepLabCutProjects\data\ToyOnlyWhite\SplitVideos\ToyOnlyExcitatory_2_26_25_S1Y_Doc.mp4",
             r"C:\DeepLabCutProjects\data\ToyOnlyWhite\SplitVideos\ToyOnlyExcitatory_2_26_25_S1Y_Dopey.mp4",
             r"C:\DeepLabCutProjects\data\ToyOnlyWhite\SplitVideos\ToyOnlyExcitatory_2_26_25_S1Y_Grumpy.mp4",
             r"C:\DeepLabCutProjects\data\ToyOnlyWhite\SplitVideos\ToyOnlyExcitatory_2_26_25_S1Y_Sneezy.mp4",
             r"C:\DeepLabCutProjects\data\ToyOnlyWhite\SplitVideos\ToyOnlyExcitatory_2_26_25_S2Y_Bashful.mp4",
             r"C:\DeepLabCutProjects\data\ToyOnlyWhite\SplitVideos\ToyOnlyExcitatory_2_26_25_S2Y_Happy.mp4"]

In [ ]:
deeplabcut.add_new_videos(config_path, new_videos, copy_videos=False, extract_frames=False)


### Check the Backend (TensorFlow or PyTorch)

In [ ]:
try:
    from tensorflow.python.client import device_lib
    print("Using TensorFlow backend.")
except ImportError:
    try:
        import torch
        print("Using PyTorch backend.")
    except ImportError:
        print("No supported backend found. Ensure TensorFlow or PyTorch is installed.")

## Configure Project

Now that a new project has been created with a specific directory structure and configuration file, we need to tweak some parameters to tailor the bodyparts we want to track:

In [ ]:
config_path = r'C:\DeepLabCutProjects\DLC-WhiteAnimals-Atanu-2025-07-23\config.yaml'
print(config_path)


In [ ]:
import webbrowser
webbrowser.open(config_path)
print('Please edit bodyparts list to be tracked')

In [ ]:
with open(config_path, 'r') as file:
    config_content = file.read()

Once happy with all ```bodyparts```, ```skeleton:``` and ```numframes2pick:``` settings, start extracting frames to label:


## Select Frames to Label

In [ ]:
deeplabcut.extract_frames(
    config_path, mode='automatic', 
    algo='uniform', 
    crop='GUI', # <--- THIS IS KEY: Tells DLC you want to crop
    userfeedback=True # <--- THIS IS KEY: Ensures an interactive GUI for selection
)

# deeplabcut.extract_frames(config_path, "manual")

## Label Frames (Do it in GUI)

In [ ]:
deeplabcut.label_frames(config_path)


You can plot your labeled frames to check your annotation accuracy.

In [ ]:
deeplabcut.check_labels(config_path)

## Create Training Dataset

In [ ]:
deeplabcut.create_training_dataset(config_path)


## Train The Network

In [ ]:
deeplabcut.train_network(
    config_path,
    shuffle=3,
    trainingsetindex=0,
    device="cuda:0",
    max_snapshots_to_keep=5,
    displayiters=100,
    save_epochs=5,
    epochs=200,
)


## Evaluate the Trained Network

In [ ]:
deeplabcut.evaluate_network(config_path, Shuffles=[3], plotting=True)

## Analyze new Videos

#### Test videos

In [ ]:
videolist = [r"C:\DeepLabCutProjects\data\Test\ToyOnly_2_25_25_S1P_Dallas_trimmed.mp4",
            r"C:\DeepLabCutProjects\data\Test\ToyOnlyInhibitory_2_27_25_S3P_J_trimmed.mp4",
            r"C:\DeepLabCutProjects\data\Test\ToyOnlyInhibitory_2_27_25_S3P_K_trimmed.mp4"]
# destfolder = r"C:\DeepLabCutProjects\data\FoodLight\DlcDataPytorch"

deeplabcut.analyze_videos(config_path, videos=videolist, shuffle=3, gputouse="cuda:0", save_as_csv=True)


#### All videos in a directory

In [ ]:
import os

# base data path
base_path = r"C:\DeepLabCutProjects\data"

# List of subfolders to process
conditions = ['ToyOnlyWhite', 'ToyLightWhite']

# Loop over all conditions
for condition in conditions:
    print(f"\nProcessing: {condition}")

    video_dir = os.path.join(base_path, condition, "SplitVideos")
    destfolder = os.path.join(base_path, condition, "DlcDataPytorch")
    os.makedirs(destfolder, exist_ok=True)

    # Get list of video files
    videolist = [
        os.path.join(video_dir, f)
        for f in os.listdir(video_dir)
        if f.endswith(('.mp4', '.avi'))
    ]

    # Skip if no videos
    if not videolist:
        print(f"No videos found in {video_dir}, skipping.")
        continue

    # Analyze
    print(f"Analyzing {len(videolist)} videos, saving to {destfolder}")
    deeplabcut.analyze_videos(
        config_path,
        videos=videolist,
        shuffle=3,
        gputouse="cuda:0",
        save_as_csv=True,
        destfolder=destfolder
    )

print("\nAll batches submitted for analysis.")


## Filter Pose Data

#### Test videos

In [ ]:
deeplabcut.filterpredictions(config_path, videolist, shuffle=3, filtertype='median', p_bound=0.05)

#### All videos in a directory

In [ ]:
import os

# base data path
base_path = r"C:\DeepLabCutProjects\data"

# List of subfolders to process
conditions = ['ToyOnlyWhite']

# Loop over all conditions
for condition in conditions:
    print(f"\nProcessing: {condition}")

    video_dir = os.path.join(base_path, condition, "SplitVideos")

    # Get list of video files
    videolist = [
        os.path.join(video_dir, f)
        for f in os.listdir(video_dir)
        if f.endswith(('.mp4', '.avi'))
    ]

    # Skip if no videos
    if not videolist:
        print(f"No videos found in {video_dir}, skipping.")
        continue

    # Analyze
    deeplabcut.filterpredictions(
        config_path,
        videolist,
        shuffle=3,
        save_as_csv=True
    )

print("\nAll batches submitted for analysis.")

In [ ]:
deeplabcut.analyzeskeleton(config_path, videolist, videotype='.mp4', shuffle=1, trainingsetindex=0, save_as_csv=False, destfolder=None)

## Create labeled videos

In [ ]:
import os

# video_dir = r"C:\DeepLabCutProjects\data\FoodOnly\SplitVideos"
# videolist = [os.path.join(video_dir, f) for f in os.listdir(video_dir) if f.endswith(('.mp4', '.avi'))]

# videolist = [r"C:\DeepLabCutProjects\DLC-Atanu-2025-06-10\videos\FoodOnly\FoodOnly_8_28_24_S3P_Cyan_Trial1.mp4"]

# deeplabcut.create_labeled_video(config_path, videolist, draw_skeleton=False, filtered=True)

deeplabcut.create_labeled_video(
    config_path,
    videolist,
    shuffle=3,
    filtered=True,
    fastmode=False,
    save_frames=False,
    displayedbodyparts=['Head', 'Neck', 'Midback', 'Lowerback', 'Tailbase']
)


### Plot Trajectories

In [ ]:
deeplabcut.plot_trajectories(config_path, videolist)

# Optional Active Learning -> Network Refinement

Load new videos to analyze and/or merge to the project:

## Extract outlier frames

Now this is the interesting part. Instead of including more videos to the project directly, and extracting frames as usual with kmeans, we are taking advantage of the previous model to tell us what frames exactly to label. This active learning step helps us recognize the shortcomings of our model and correct it in a targeted manner.

In [ ]:
deeplabcut.extract_outlier_frames(config_path, videolist, outlieralgorithm='uncertain')

## Refine Labels: Augmentation of the Training Dataset

Now that we have extracted new frames, we need to go back and start labeling. Instead of starting from the beginning, though, we are provided the model predictions and have to drag and drop them in place. **Note:** Make sure to remove labels that are not visible, the model will often guess the expected position based on learned geometric constraints. 

In [ ]:
deeplabcut.refine_labels(config_path)

You can again plot your labeled frames to check annotation accuracy.

In [ ]:
deeplabcut.check_labels(config_path)

At this point you could get an error message like [this](https://github.com/DeepLabCut/DeepLabCut/issues/232) telling you that saving the video path failed. In this case, you need to add the new video paths manually for DLC to include these in the new training set. You can either add them by hand, writing in the config.yaml file in the same format as the first video paths (see [here](https://github.com/DeepLabCut/DeepLabCut/issues/663#issuecomment-619274975)), or you can run the following command to add the list of videos to your config file:

If the permission error persists, try starting a new anaconda terminal as administrator (right click > run as administrator) and then starting jupyter notebook with elevated privileges.

## Merge Datasets

After refining all outlier frames extracted above, merge the datasets to combine old and new labels in your project. 

In [ ]:
deeplabcut.merge_datasets(config_path)

**Note:** Make sure that the new videos have been included in the config.yaml file without permission issues (see above).

### Re-Train Network

When training a new model with an expanded dataset, you could either choose to start fresh with new data, or use the previous model as pre-trained network for your next model. Although not yet extensively verified, lets belief that transfer learning at least won't harm the new model. 

### Example of own pre-trained model
init_weights: D:\FacialExpression\old-DLC-Project\dlc-models\iteration-0\DLCApr14-trainset95shuffle1\train\snapshot-1030000

## Start over again...